In [1]:
# !pip install websockets

In [2]:
import pymssql
from vosk import Model, KaldiRecognizer, SetLogLevel
import wave
import json
import asyncio
import websockets

In [3]:
# cpu approach
model_path = '/mnt/share/audio_call/model_v0/model'
model = Model(model_path)

In [4]:
def connect_sql():

    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'
    
    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()
    
    return pymssql.connect(
        server=sql_server,
        user=sql_login,
        password=sql_pass,
        database=sql_name,
    )

In [5]:
def get_file(source_id):
    
    conn = connect_sql()
    cursor = conn.cursor()
    
    # middle date
    sql_query = "select"
    sql_query += " CAST(AVG(CAST(record_date-0.1 AS FLOAT)) AS DATETIME),"
    sql_query += " CAST(AVG(CAST(record_date+0.1 AS FLOAT)) AS DATETIME)"
    sql_query += " from queue"
    sql_query += " where source_id = '"+source_id+"' and duration>60;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        min_time = str(row[0])[:19]
        max_time = str(row[1])[:19]
    
    # file name and path from queue
    sql_query = "select top 1 filepath, filename from queue "
    sql_query += "where source_id = '"+source_id+"' "
    sql_query += " and record_date > '" + min_time + "' "
    sql_query += " and record_date < '" + max_time + "' "
    sql_query += " order by record_date, filename;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        original_file_path = row[0]
        original_file_name = row[1]
        
    return original_file_path, original_file_name

In [6]:
#original_file_path, original_file_name = get_file('1')
#original_file_path, original_file_name

### MRM records

In [8]:
original_file_path, original_file_name = get_file('2')

In [9]:
original_file_path, original_file_name

('/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/',
 'a2021-08-17t21:22:25b_c9531051950d_e9180117047f_g1629224543.2646348h-in.wav')

In [32]:
def cpu_approach(model, original_file_path, original_file_name):
    # read file
    wf = wave.open(original_file_path + original_file_name, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    phrases_count = 0
    confidences = []
    phrases = []
    while True:

        conf_score = []

        data = wf.readframes(4000)
        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            accept = json.loads(rec.Result())
            print('*', accept)
            if accept['text'] != '':

                accept_start = str(accept['result'][0]['start'])
                accept_end = accept['result'][-1:][0]['end']
                accept_text = str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                confidences.append(sum(conf_score)/len(conf_score))
                print(accept_start, accept_text)
                phrases.append({
                    'time': accept_start,
                    'conf': conf_mid,
                    'text': accept_text
                })
                phrases_count += 1
    print('end')
    return phrases

In [33]:
phrases_cpu = cpu_approach(model, original_file_path, original_file_name)
phrases_cpu

* {'text': ''}
* {'text': ''}
* {'result': [{'conf': 0.279026, 'end': 14.578151, 'start': 14.311006, 'word': 'алло'}], 'text': 'алло'}
14.311006 алло
* {'result': [{'conf': 0.932752, 'end': 15.66, 'start': 15.450601, 'word': 'да'}], 'text': 'да'}
15.450601 да
* {'result': [{'conf': 0.397711, 'end': 21.403667, 'start': 21.077715, 'word': 'а'}], 'text': 'а'}
21.077715 а
* {'text': ''}
* {'text': ''}
* {'text': ''}
* {'text': ''}
* {'text': ''}
* {'result': [{'conf': 0.518808, 'end': 52.319963, 'start': 52.08, 'word': 'ну'}], 'text': 'ну'}
52.08 ну
* {'text': ''}
* {'text': ''}
* {'result': [{'conf': 0.71517, 'end': 63.868411, 'start': 63.66, 'word': 'ну'}, {'conf': 0.405397, 'end': 64.17, 'start': 63.868411, 'word': 'остро'}, {'conf': 0.538667, 'end': 64.59, 'start': 64.172402, 'word': 'порезал'}, {'conf': 0.991365, 'end': 64.77, 'start': 64.590732, 'word': 'мне'}, {'conf': 1.0, 'end': 65.1, 'start': 64.77, 'word': 'тоже'}, {'conf': 0.950339, 'end': 65.248799, 'start': 65.1, 'word': 'не'

* {'result': [{'conf': 0.997097, 'end': 146.43, 'start': 146.16, 'word': 'это'}, {'conf': 1.0, 'end': 146.61, 'start': 146.43, 'word': 'уже'}, {'conf': 0.425406, 'end': 147.15, 'start': 146.61, 'word': 'знакомо'}, {'conf': 1.0, 'end': 147.51, 'start': 147.154014, 'word': 'честно'}, {'conf': 1.0, 'end': 147.99, 'start': 147.51, 'word': 'говоря'}], 'text': 'это уже знакомо честно говоря'}
146.16 это уже знакомо честно говоря
* {'result': [{'conf': 0.609029, 'end': 149.429883, 'start': 149.28, 'word': 'ну'}, {'conf': 0.735021, 'end': 149.850088, 'start': 149.429883, 'word': 'смотрите'}, {'conf': 0.650347, 'end': 150.059971, 'start': 149.850088, 'word': 'вот'}], 'text': 'ну смотрите вот'}
149.28 ну смотрите вот
* {'result': [{'conf': 0.925545, 'end': 152.010117, 'start': 151.65, 'word': 'ну'}, {'conf': 0.408839, 'end': 152.278901, 'start': 152.010117, 'word': 'мин'}, {'conf': 0.569003, 'end': 152.91, 'start': 152.278901, 'word': 'минимальная'}, {'conf': 0.881411, 'end': 153.411812, 'start'

* {'result': [{'conf': 0.904115, 'end': 218.7, 'start': 218.52, 'word': 'я'}, {'conf': 0.999985, 'end': 219.210015, 'start': 218.846323, 'word': 'приеду'}, {'conf': 0.999393, 'end': 219.33, 'start': 219.210132, 'word': 'не'}, {'conf': 0.998585, 'end': 219.839971, 'start': 219.33, 'word': 'проблема'}, {'conf': 1.0, 'end': 220.29, 'start': 219.84, 'word': 'просто'}, {'conf': 0.990218, 'end': 221.52, 'start': 220.49458, 'word': 'ваше'}, {'conf': 0.990218, 'end': 221.97, 'start': 221.52, 'word': 'решение'}, {'conf': 1.0, 'end': 222.09, 'start': 221.97, 'word': 'по'}, {'conf': 1.0, 'end': 222.54, 'start': 222.09, 'word': 'поводу'}], 'text': 'я приеду не проблема просто ваше решение по поводу'}
218.52 я приеду не проблема просто ваше решение по поводу
* {'result': [{'conf': 1.0, 'end': 225.87, 'start': 225.54, 'word': 'ну'}, {'conf': 0.930085, 'end': 226.071533, 'start': 225.87, 'word': 'да'}, {'conf': 1.0, 'end': 226.219512, 'start': 226.071533, 'word': 'да'}, {'conf': 1.0, 'end': 226.35, '

* {'result': [{'conf': 0.888974, 'end': 288.201562, 'start': 287.734541, 'word': 'все'}, {'conf': 0.65388, 'end': 288.555615, 'start': 288.201562, 'word': 'все'}, {'conf': 0.546611, 'end': 289.2, 'start': 288.6, 'word': 'добро'}], 'text': 'все все добро'}
287.734541 все все добро
* {'result': [{'conf': 0.310789, 'end': 290.857383, 'start': 290.767383, 'word': 'до'}, {'conf': 0.37861, 'end': 291.27, 'start': 290.858701, 'word': 'свиданий'}], 'text': 'до свиданий'}
290.767383 до свиданий
end


[{'time': '14.311006', 'conf': '0.279026', 'text': 'алло'},
 {'time': '15.450601', 'conf': '0.932752', 'text': 'да'},
 {'time': '21.077715', 'conf': '0.397711', 'text': 'а'},
 {'time': '52.08', 'conf': '0.518808', 'text': 'ну'},
 {'time': '63.66',
  'conf': '0.8009324999999998',
  'text': 'ну остро порезал мне тоже не только перфоратором'},
 {'time': '68.73', 'conf': '0.773905', 'text': 'потому что да а'},
 {'time': '77.01', 'conf': '0.8266215', 'text': 'нет на'},
 {'time': '80.01',
  'conf': '0.8868539',
  'text': 'ой вот как раз фортуна это будет палящего все жуткое'},
 {'time': '86.4',
  'conf': '0.8760784285714285',
  'text': 'нет перфоратором просто подмести нас будут здесь'},
 {'time': '90.81',
  'conf': '0.6376075714285714',
  'text': 'со мной мною пыли и не будет'},
 {'time': '95.01',
  'conf': '0.9160048571428571',
  'text': 'хотя то там что вас бетон или'},
 {'time': '99.66', 'conf': '0.846343', 'text': 'ан'},
 {'time': '105.09', 'conf': '0.9580553333333334', 'text': 'ну поня

In [47]:
#async def gpu_approach(original_file_path, original_file_name):
server_ip = '10.2.5.212'
uri = 'ws://' + server_ip + ':2700'
async with websockets.connect(uri) as websocket:
    wf = open(original_file_path + original_file_name, "rb")
    while True:
        data = wf.read(8000)
        if len(data) == 0:
            break
        await websocket.send(data)
        accept = json.loads(await websocket.recv())
        print('*', accept)
        if len(accept)>1 and accept['text'] != '':
            file_data.append(accept['text'])
            textfile.write(str(accept['text'])+'\n')
        else:
            [[key, value]] = accept.items()
            if key == 'text' and len(value):
                print('===', value)
    await websocket.send('{"eof" : 1}')
    print(await websocket.recv())
            
#        return accept
#asyncio.get_event_loop().run_until_complete(
#    gpu_approach(original_file_path, original_file_name)
#)

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}


ConnectionClosedError: code = 1011 (unexpected error), no reason